# Projet final
## Réalisé par : 
## OUPRAXAY Philippe
## TOURE Almamy Moustapha
### Master 2 SEP
### Année : 2020/2021

## importation des bibliothèques et modules

In [2]:
import findspark
findspark.init("C:\spark")
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf 
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

In [3]:
# instantiation de spark 
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("Projet_final")\
                    .getOrCreate()

## Questions :  
## à partir d’un spark session réaliser ces requêtes en DSL et en SQL


## 0-creation des tables

In [4]:
# Author
T1= [("07890", "Jean Paul Sartre"), ("05678", "Pierre de Ronsard")]
RDD1 = spark.sparkContext.parallelize(T1)
Author = RDD1.toDF(["aid", "name"])
Author.show()
Author.createOrReplaceTempView("Author_SQL")

+-----+-----------------+
|  aid|             name|
+-----+-----------------+
|07890| Jean Paul Sartre|
|05678|Pierre de Ronsard|
+-----+-----------------+



In [5]:
# Book
T2 = [("0001", "L'existentialisme est un humanisme", "Philosophie"), ("0002", "Huis clos. Suivi de Les Mouches ", "Philosophie"),
     ("0003", "Mignonne allons voir si la rose", "Poeme"), ("0004", "Les Amours", "Poème")]
RDD2 = spark.sparkContext.parallelize(T2)
Book = RDD2.toDF(["bid", "title", "category"])
Book.show()
Book.createOrReplaceTempView("Book_SQL")

+----+--------------------+-----------+
| bid|               title|   category|
+----+--------------------+-----------+
|0001|L'existentialisme...|Philosophie|
|0002|Huis clos. Suivi ...|Philosophie|
|0003|Mignonne allons v...|      Poeme|
|0004|          Les Amours|      Poème|
+----+--------------------+-----------+



In [6]:
# Student
T3 = [("S15", "toto", "Math" ), ("S16", "popo", "Eco"), ("S17", "fofo", "Mécanique") ]
RDD3 = spark.sparkContext.parallelize(T3)
Student = RDD3.toDF(["sid", "sname", "dept"])
Student.show()
Student.createOrReplaceTempView("Student_SQL")

+---+-----+---------+
|sid|sname|     dept|
+---+-----+---------+
|S15| toto|     Math|
|S16| popo|      Eco|
|S17| fofo|Mécanique|
+---+-----+---------+



In [7]:
# Write
T4 = [("07890", "0001"), ("07890", "0002"), ("05678", "0003"), ("05678", "0003")]
RDD4 = spark.sparkContext.parallelize(T4)
Write = RDD4.toDF(["aid", "bid"])
Write.show()
Write.createOrReplaceTempView("Write_SQL")

+-----+----+
|  aid| bid|
+-----+----+
|07890|0001|
|07890|0002|
|05678|0003|
|05678|0003|
+-----+----+



In [8]:
# borrow
T5 = [("S15", "0003", "02-01-2020", "01-02-2020"), ("S15", "0002", "13-06-2020", "null"),
     ("S15", "0001", "13-06-2020", "13-10-2020"), ("S16", "0002", "24-01-2020", "24-01-2020"),
     ("S17", "0001", "12-04-2020", "01-07-2020")]
RDD5 = spark.sparkContext.parallelize(T5)
Borrow = RDD5.toDF(["sid", "bid", "checkout_time", "return_time"])
Borrow.show()
Borrow.createOrReplaceTempView("Borrow_SQL")

+---+----+-------------+-----------+
|sid| bid|checkout_time|return_time|
+---+----+-------------+-----------+
|S15|0003|   02-01-2020| 01-02-2020|
|S15|0002|   13-06-2020|       null|
|S15|0001|   13-06-2020| 13-10-2020|
|S16|0002|   24-01-2020| 24-01-2020|
|S17|0001|   12-04-2020| 01-07-2020|
+---+----+-------------+-----------+



## 1-Trouver les titres de tous les livres que l'étudiant sid='S15' a emprunté.

In [9]:
# SQL
spark.sql("""
SELECT title as titre
FROM Borrow_SQL 
RIGHT JOIN Book_SQL 
    on Borrow_SQL.bid = Book_SQL.bid 
where sid == 'S15'
""").show()
print(" ######Voici les titres des tous les livres empruntés par l'étudiant S15 ######")

+--------------------+
|               titre|
+--------------------+
|Huis clos. Suivi ...|
|Mignonne allons v...|
|L'existentialisme...|
+--------------------+

 ######Voici les titres des tous les livres empruntés par l'étudiant S15 ######


In [10]:
# DSL
Borrow.join(Book, Borrow.bid == Book.bid)\
     .select(col("title").alias("titre"))\
     .filter(col("sid") == "S15")\
     .show()

+--------------------+
|               titre|
+--------------------+
|Huis clos. Suivi ...|
|Mignonne allons v...|
|L'existentialisme...|
+--------------------+



## 2-Trouver les titres de tous les livres qui n'ont jamais été empruntés par un étudiant

In [11]:
# SQL
spark.sql(""" 
select  B1.title as titre 
    from Book_SQL as B1
    left outer join Borrow_SQL as B2
    on B1.bid == B2.bid
    where sid is Null
""").show()
print("#####Voici les titres de tous les livres n'ayant jamais été empruntés par un étudiant")

+----------+
|     titre|
+----------+
|Les Amours|
+----------+

#####Voici les titres de tous les livres n'ayant jamais été empruntés par un étudiant


In [12]:
#DSL

Book.join(Borrow, Borrow.bid == Book.bid, how = 'left_outer')\
     .filter(F.col("sid").isNull())\
     .select(col("title").alias("titre"))\
     .show()

+----------+
|     titre|
+----------+
|Les Amours|
+----------+



## 3-Trouver tous les étudiants qui ont emprunté le livre bid=’0002’

In [13]:
# SQL

spark.sql(""" 
select sname as Nom
from Book_SQL as B1
    left outer join Borrow_SQL as B2 on B1.bid == B2.bid
    left outer join Student_SQL as B3 on B2.sid == B3.sid
    where B2.bid =="0002"  
    """).show()
print(" ####Voici les noms des étudiants ayant emprunté le livre 00002####")

+----+
| Nom|
+----+
|popo|
|toto|
+----+

 ####Voici les noms des étudiants ayant emprunté le livre 00002####


In [14]:
# DSL
Student.join(Borrow, Borrow.sid == Student.sid, how = 'left_outer')\
     .filter(F.col("bid") == "0002")\
     .select(col("sname").alias("Nom"))\
     .show()

+----+
| Nom|
+----+
|popo|
|toto|
+----+



## 4-Trouver les titres de tous les livres empruntés par des étudiants en Mécanique (département Mécanique)

In [15]:
# SQL
spark.sql("""
select title as Titre
from Book_SQL as B1
left outer join Borrow_SQL as B2
    on B1.bid == B2.bid 
left outer join Student_SQL as B3
    on B2.sid== B3.sid
where B3.dept == 'Mécanique'
""").show()
print("####Voici les titres de tous les livres empruntés par des étudiants en Mécanique####")

+--------------------+
|               Titre|
+--------------------+
|L'existentialisme...|
+--------------------+

####Voici les titres de tous les livres empruntés par des étudiants en Mécanique####


In [16]:
# DSL
Book\
    .join(Borrow,['bid'])\
    .join(Student,['sid'])\
    .filter(F.col('dept')=='Mécanique')\
    .select(col('title').alias("Titre"))\
    .show()



+--------------------+
|               Titre|
+--------------------+
|L'existentialisme...|
+--------------------+



## 5-Trouver les étudiants qui n’ont jamais emprunté de livre.

In [17]:
# SQL

spark.sql(""" 
select sname as Nom
from Book_SQL as B1
left outer join Borrow_SQL as B2
    on B1.bid == B2.bid
left outer join Student_SQL as B3
    on B2.sid == B3.sid
where B1.bid is Null  
""").show()
print("#### Voici Les étudiants n'ayant jamais emprunté de livres ####")

+---+
|Nom|
+---+
+---+

#### Voici Les étudiants n'ayant jamais emprunté de livres ####


In [18]:
# DSL
Student.join(Borrow, Borrow.sid == Student.sid, how = 'left_outer')\
     .filter(F.col("bid").isNull())\
     .select(col("sname").alias("Nom"))\
     .show()


+---+
|Nom|
+---+
+---+



## 6- Déterminer l’auteur qui a écrit le plus de livres.

In [19]:
# SQL

spark.sql("""
SELECT  name as Nom, count(distinct(bid)) as nb_livre_ecrit
FROM Author_SQL 
RIGHT JOIN Write_SQL 
    on Author_SQL.aid = Write_SQL.aid 
group by name
order by nb_livre_ecrit desc limit(1)


""").show()
print(" ##### Voici l'auteur qui a écrit le plus de livres #####")

+----------------+--------------+
|             Nom|nb_livre_ecrit|
+----------------+--------------+
|Jean Paul Sartre|             2|
+----------------+--------------+

 ##### Voici l'auteur qui a écrit le plus de livres #####


In [20]:
# DSL
Author.join(Write, ["aid"])\
        .distinct()\
        .groupBy("name")\
        .agg(F.count("bid").alias("nombre"))\
        .sort(F.col("nombre").desc())\
        .select(F.first("name").alias("Nom"),F.first("nombre").alias("nb_livre_ecrit"))\
        .show()

+----------------+--------------+
|             Nom|nb_livre_ecrit|
+----------------+--------------+
|Jean Paul Sartre|             2|
+----------------+--------------+



## 7- Déterminer les personnes qui n’ont pas encore rendu les livres.

In [21]:
# SQL 
spark.sql("""
select sname as Nom
from Student_SQL 
join Borrow_SQL on Student_SQL.sid = Borrow_SQL.sid
where Borrow_SQL.return_time = 'null' 
""").show()
print("####Voici les personnes n'ayant pas rendu encore les livres ####")


+----+
| Nom|
+----+
|toto|
+----+

####Voici les personnes n'ayant pas rendu encore les livres ####


In [22]:
#DSL
Borrow.filter(Borrow.return_time =='null')\
    .join(Student, Borrow.sid==Student.sid)\
    .select(col("sname").alias("Nom"))\
    .show()


+----+
| Nom|
+----+
|toto|
+----+



## 8- Créer une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois,  sinon 0. 

In [23]:
#sql
spark.sql("""
    select *,
    case
        when (months_between(to_date(`return_time`, 'dd-MM-yyyy'),
                   to_date(`checkout_time`, 'dd-MM-yyyy')) >= 3) then 1
        else 0
    end as `plus que 3 mois`
    from Borrow_SQL
""").show()

+---+----+-------------+-----------+---------------+
|sid| bid|checkout_time|return_time|plus que 3 mois|
+---+----+-------------+-----------+---------------+
|S15|0003|   02-01-2020| 01-02-2020|              0|
|S15|0002|   13-06-2020|       null|              0|
|S15|0001|   13-06-2020| 13-10-2020|              1|
|S16|0002|   24-01-2020| 24-01-2020|              0|
|S17|0001|   12-04-2020| 01-07-2020|              0|
+---+----+-------------+-----------+---------------+



In [24]:
# DSL

Borrow2=Borrow.withColumn("debut", F.to_date(F.col("checkout_time"), "dd-MM-yyyy"))\
    .withColumn("retour", F.to_date(F.col("return_time"), "dd-MM-yyyy"))\
    .withColumn("temps", F.months_between(F.col("retour"), F.col("debut")))\
    .withColumn("plus_de_3_mois", (F.when(F.col("temps")>=3, 1).otherwise(0)))\
    .select("sid", "bid", "checkout_time", "return_time", "plus_de_3_mois", "temps")
Borrow2.show()

+---+----+-------------+-----------+--------------+----------+
|sid| bid|checkout_time|return_time|plus_de_3_mois|     temps|
+---+----+-------------+-----------+--------------+----------+
|S15|0003|   02-01-2020| 01-02-2020|             0|0.96774194|
|S15|0002|   13-06-2020|       null|             0|      null|
|S15|0001|   13-06-2020| 13-10-2020|             1|       4.0|
|S16|0002|   24-01-2020| 24-01-2020|             0|       0.0|
|S17|0001|   12-04-2020| 01-07-2020|             0|2.64516129|
+---+----+-------------+-----------+--------------+----------+



In [25]:
# exportation du resultat
Borrow2.toPandas().to_csv("contention/dtaa.csv", header = True)


## 9-déterminer les livres qui n’ont jamais été empruntés. identique à la 2 eme question

In [26]:
# SQL
spark.sql(""" select B1.title
    from Book_SQL as B1
    left outer join Borrow_SQL as B2
    on B1.bid == B2.bid
    where sid is Null""").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



In [27]:
# DSL
Book.join(Borrow, Borrow.bid == Book.bid, how = 'left_outer')\
     .filter(F.col("sid").isNull())\
     .select( "title")\
     .show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+

